In [1]:
import re

input_files = ['ALL.wgs.integrated_sv_map_v2.20130502.svs.genotypes.vcf']

dataset = {}
nan_lines = []
for input_file in input_files:
    with open(input_file,'r') as f:
        n=0
        for line in f:
            n += 1
            if line[0] not in ['#',' ','\n','']:


                data = line.split('\t')
                chromosome   = data[0]
                if chromosome not in dataset:
                    dataset.update({chromosome:[]})
                start = int(data[1])
                try:
                    end = int(re.search(';END=\w+', line).group().replace(';END=',''))
                    sv_type = re.search('SVTYPE=\w+', line).group().replace('SVTYPE=','')
                    dataset[chromosome].append([start,end,sv_type])
                except Exception:
                    sv_type = re.search('SVTYPE=\w+', line).group().replace('SVTYPE=','')
                    dataset[chromosome].append([start,'NaN',sv_type])
                    nan_lines.append(line.split('0|0')[0])

In [2]:
print(len(dataset))
print(len(nan_lines))

23
16631


In [3]:
nan_lines[0]

'1\t645710\tALU_umary_ALU_2\tA\t<INS:ME:ALU>\t.\t.\tTSD=null;SVTYPE=ALU;MEINFO=AluYa4_5,1,223,-;SVLEN=222;CS=ALU_umary;AC=35;AF=0.00698882;NS=2504;AN=5008;EAS_AF=0.0069;EUR_AF=0.0189;AFR_AF=0.0;AMR_AF=0.0072;SAS_AF=0.0041;SITEPOST=0.9998\tGT\t'

In [3]:
##Basic analysis
from collections import Counter

CHR_NAME = [str(x) for x in range(1,23)]+['X','Y']
sv_type_by_chr = {}
all_types = set()

distance = [] #distance between breaks

for c in CHR_NAME:
    try:
        print(c,len(dataset[c]))
        sv_type_by_chr.update({c:Counter()})
        for item in dataset[c]:
            sv_type = item[-1]
            sv_type_by_chr[c].update({sv_type})
            all_types.add(sv_type)
            
            try:
                distance.append(item[1]-item[0])
            except Exception as e:
                pass
                #print(e, item[1],item[0])
    except KeyError:
        pass
        

1 4671
2 5642
3 4811
4 4780
5 4425
6 4187
7 4200
8 3681
9 3001
10 3126
11 3375
12 3299
13 2485
14 2097
15 1867
16 2062
17 1926
18 2005
19 1621
20 1569
21 877
22 848
X 2263


In [4]:
all_types = sorted(list(all_types))
header = '#Chr'
for item in all_types:
    header += '\t{}'.format(item)
print(header)

for c in CHR_NAME:
    line = '{}'.format(c)

    for item in all_types:
        try:
            line += '\t{}'.format(sv_type_by_chr[c][item])
        except KeyError:
            line += '\t{}'.format(0)
    print(line)
    


#Chr	ALU	CNV	DEL	DEL_ALU	DEL_HERV	DEL_LINE1	DEL_SVA	DUP	INS	INV	LINE1	SVA
1	949	162	2757	49	0	5	0	353	9	68	234	85
2	1125	210	3374	101	0	3	0	403	15	63	284	64
3	958	163	2869	82	0	2	1	384	13	58	230	51
4	964	171	2766	110	0	8	1	399	18	54	262	27
5	883	176	2546	93	1	2	1	418	14	51	202	38
6	859	162	2430	90	0	3	1	340	10	53	186	53
7	751	194	2498	72	0	5	0	403	5	61	168	43
8	676	165	2213	73	0	2	0	311	12	53	146	30
9	525	171	1793	65	0	1	1	238	7	42	122	36
10	593	139	1784	55	0	3	2	308	8	41	146	47
11	604	138	2006	69	0	2	0	300	8	35	166	47
12	649	125	1895	51	0	3	0	340	10	42	153	31
13	533	98	1366	61	0	2	0	233	6	30	129	27
14	404	69	1226	39	0	4	0	190	4	23	103	35
15	387	90	1082	28	0	2	0	165	1	16	68	28
16	242	143	1330	28	0	0	1	219	3	16	53	27
17	303	89	1213	28	0	0	0	185	5	22	42	39
18	374	102	1224	40	0	1	0	157	2	16	77	12
19	150	98	1143	15	0	0	0	144	2	9	24	36
20	237	52	970	38	0	2	1	155	3	17	58	36
21	192	26	517	13	0	1	0	68	4	11	38	7
22	96	55	573	9	0	0	0	79	4	5	7	20
X	294	131	1400	29	0	5	0	233	5	0	150	16
Y	0	0	0	0	0

In [5]:
sv_type_by_chr['1']['ALU']

949

In [6]:
distance = sorted(distance)
distance_distribution = Counter()

for item in distance:
    interval = round(item/1000)
    distance_distribution.update({interval})
for k in distance_distribution:
    print('{}\t{}'.format(k,distance_distribution[k]))

0	9596
1	7496
2	5149
3	3551
4	2927
5	2540
6	1873
7	1493
8	1184
9	1013
10	950
11	889
12	719
13	590
14	556
15	490
16	425
17	373
18	328
19	324
20	314
21	264
22	246
23	248
24	225
25	234
26	220
27	194
28	189
29	199
30	192
31	173
32	162
33	145
34	150
35	143
36	132
37	143
38	155
39	133
40	135
41	111
42	127
43	111
44	126
45	111
46	108
47	88
48	119
49	108
50	95
51	97
52	92
53	88
54	76
55	95
56	66
57	80
58	91
59	70
60	80
61	72
62	70
63	55
64	67
65	68
66	48
67	54
68	54
69	65
70	53
71	71
72	43
73	40
74	37
75	44
76	56
77	52
78	43
79	45
80	48
81	41
82	60
83	50
84	44
85	40
86	36
87	35
88	34
89	37
90	31
91	42
92	26
93	37
94	39
95	25
96	27
97	35
98	32
99	23
100	38
101	26
102	28
103	23
104	27
105	21
106	14
107	33
108	30
109	21
110	23
111	12
112	19
113	23
114	17
115	19
116	38
117	20
118	31
119	28
120	14
121	30
122	28
123	14
124	28
125	18
126	14
127	16
128	27
129	22
130	16
131	29
132	21
133	20
134	15
135	21
136	21
137	31
138	20
139	16
140	8
141	25
142	16
143	18
144	14
145	12
146	16
147	14
148	19
149	11
15

In [8]:
from datetime import date
today = date.today()

# input and output file
input_file = input_files[0]
output_file  = 'dataset_1000G_{}.txt'.format(today)


comment_line = '#This file was generated using parse_1000G_CNVs_from_sv_map.ipynb'\
               ' using {} as input and {} as output.\n'.format(input_file,output_file)

header       = '##{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(
                                                                   'chr1','s1','e1','o1',
                                                                   'chr2','s2','e2','o2',
                                                                   'source','sample_name','sv_type','cancer_type')

with open(output_file, 'w') as f:
    f.write(comment_line)
    f.write(header)
    
    for c in CHR_NAME[:-1]:
        for item in dataset[c]:

            data = item
            chr1 = c
            s1   = data[0]
            e1   = data[0]
            o1   = 'NaN'
            chr2 = c
            s2   = data[1]
            e2   = data[1]
            o2   = 'NaN'
            source = '1000G'
            sample_name = 'NaN'
            sv_type     = data[2]
            cancer_type = 'Healthy Control'

            out_line = '{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(chr1,s1,e1,o1,
                                                                             chr2,s2,e2,o2,
                                                                             source,sample_name,sv_type,cancer_type)
            f.write(out_line)
        
        